In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

### PLAN

1. Pobranie danych
2. Przejrzeć co mamy
    - wielkosc data setu
    - jakie kolumny
    - jakiego typu dane
    - podejrzec probke
    - jesli dane sa we wlasciwym formacie: proste statystyki
3. Konwersja danych na odpowiedni format
4. Co z brakujacymi wartosciami?

#### Zaimportuj i przejrzyj dane

In [2]:
dane = pd.read_csv('data/DataWorkshop.csv')

In [3]:
print('Rozmiar danych: ', dane.shape)

Rozmiar danych:  (229929, 30)


In [4]:
print('Kolumny w datasecie: ')
dane.columns

Kolumny w datasecie: 


Index(['Data zdarzenia', 'Godzina', 'GPS x', 'GPS y', 'Miejscowość', 'Ulica',
       'Numer domu', 'Numer drogi', 'KM HM (Pikietaż)', 'Obszar',
       'Skrzyżowanie', 'Geometria drogi', 'Charakterystyka miejsca zdarzenia',
       'Oświetlenie', 'Warunki atmosferyczne', 'Rodzaj zdarzenia', 'Wina',
       'Rodzaj drogi', 'Nawierzchnia drogi', 'Stan nawierzchni',
       'Sygnalizacja świetlna', 'Oznakowanie poziome',
       'Liczba pasów w zdarzeniu', 'Zachowanie kierującego',
       'Zachowanie pieszego', 'Inne przyczyny', 'Liczba rannych w zdarzeniu',
       'Liczba zabitych w zdarzeniu', 'Liczba kolizji', 'Liczba wypadków'],
      dtype='object')

In [5]:
print('Typy danych: ')
dane.dtypes

Typy danych: 


Data zdarzenia                        object
Godzina                               object
GPS x                                 object
GPS y                                 object
Miejscowość                           object
Ulica                                 object
Numer domu                            object
Numer drogi                           object
KM HM (Pikietaż)                      object
Obszar                                object
Skrzyżowanie                          object
Geometria drogi                       object
Charakterystyka miejsca zdarzenia     object
Oświetlenie                           object
Warunki atmosferyczne                 object
Rodzaj zdarzenia                      object
Wina                                  object
Rodzaj drogi                          object
Nawierzchnia drogi                    object
Stan nawierzchni                      object
Sygnalizacja świetlna                 object
Oznakowanie poziome                   object
Liczba pas

In [6]:
print('Brakujace wartosci: ')

missing = dane.isnull().sum()
print(missing)

Brakujace wartosci: 
Data zdarzenia                            0
Godzina                                   0
GPS x                                     0
GPS y                                     0
Miejscowość                               0
Ulica                                 20728
Numer domu                            93884
Numer drogi                               0
KM HM (Pikietaż)                     194888
Obszar                                    1
Skrzyżowanie                         166779
Geometria drogi                       66008
Charakterystyka miejsca zdarzenia         2
Oświetlenie                               0
Warunki atmosferyczne                     1
Rodzaj zdarzenia                          0
Wina                                      0
Rodzaj drogi                              1
Nawierzchnia drogi                        1
Stan nawierzchni                          1
Sygnalizacja świetlna                     0
Oznakowanie poziome                       2
Liczba pasó

In [7]:
print('Brakujace wartosci jako procent zbioru, %: ')
index=[]
lst=[]
for item in missing.iteritems():
    index.append(item[0])
    lst.append(round((item[1]/dane.shape[0])*100,4))
missing_percentage=pd.Series(lst, index=index)
print(missing_percentage)

Brakujace wartosci jako procent zbioru, %: 
Data zdarzenia                        0.0000
Godzina                               0.0000
GPS x                                 0.0000
GPS y                                 0.0000
Miejscowość                           0.0000
Ulica                                 9.0150
Numer domu                           40.8317
Numer drogi                           0.0000
KM HM (Pikietaż)                     84.7601
Obszar                                0.0004
Skrzyżowanie                         72.5350
Geometria drogi                      28.7080
Charakterystyka miejsca zdarzenia     0.0009
Oświetlenie                           0.0000
Warunki atmosferyczne                 0.0004
Rodzaj zdarzenia                      0.0000
Wina                                  0.0000
Rodzaj drogi                          0.0004
Nawierzchnia drogi                    0.0004
Stan nawierzchni                      0.0004
Sygnalizacja świetlna                 0.0000
Oznakowanie

In [8]:
# Probka danych
dane.sample(50)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
71102,2017-05-05,09:30:00,18*37'480,50*23'548,PYSKOWICE,WYSZYŃSKIEGO,NaN,P2905S,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0
139222,2018-07-07,18:30:00,19*08'233,50*17'384,SOSNOWIEC,SZPAKÓW,NaN,G120076S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
19460,2016-05-25,12:30:00,18*53'039,50*10'451,MIKOŁÓW,GLIWICKA,3,I2,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
177168,2019-02-21,09:30:00,19*08'062,50*47'395,CZĘSTOCHOWA,BARDOWSKIEGO,20,P6627S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
96007,2017-10-08,00:05:00,19*19'411,50*44'351,ZRĘBICE,GŁÓWNA,NaN,46,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,3,2,0,1
92758,2017-09-21,14:35:00,19*01'005,50*15'079,KATOWICE,PODCHORĄŻYCH,1,G100424S,NaN,Obszar zabudowany,...,Brak,Jest,NaN,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0
23819,2016-06-25,08:10:00,19*10'041,50*12'060,MYSŁOWICE,NaN,NaN,S1,"549,9",Obszar niezabudowany,...,Brak,Nie ma,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0
93491,2017-09-22,22:10:00,18*59'301,50*07'582,TYCHY,KATOWICKA,18,P8004S,NaN,Obszar zabudowany,...,Brak,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,1,0,0,1
52999,2016-12-30,17:45:00,18*13'286,50*05'448,RACIBÓRZ,BOSACKA,5,W935,"1,3",Obszar zabudowany,...,Brak,Jest,NaN,Nieustąpienie pierwszeństwa pieszemu na przejś...,NaN,NaN,0,0,1,0
223488,2019-11-26,07:45:00,18*55'189,50*26'157,NAKŁO ŚLĄSKIE,GŁÓWNA,128,78,"81,1",Obszar zabudowany,...,Brak,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0


### Konwersja typu danych

#### Data i czas

In [9]:
# Utworzenie kopii zbioru danych. Dalsza konwersja odbywac sie bedzie na kopii danych
df=dane.copy()

In [10]:
# Data zdarzenia i Godzina scalone do jednej kolumny
print('kolumna DATA ZDARZENIA: ', df['Data zdarzenia'].dtypes)
print('kolumna GODZINA: ', df['Data zdarzenia'].dtypes)
df['data_full'] = df['Data zdarzenia'] + ' ' + df['Godzina']
df.sample(10)

kolumna DATA ZDARZENIA:  object
kolumna GODZINA:  object


,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
149564,2018-09-13,10:15:00,18*48'230,50*22'370,ZABRZE,LUDWIKI WAWRZYŃSKIEJ,1,G140005S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018-09-13 10:15:00
138501,2018-07-03,09:00:00,19*01'513,50*19'327,SIEMIANOWICE ŚLĄSKIE,HENRYKA KRUPANKA,83,94,NaN,Obszar zabudowany,...,Jest,2.0,"Zmęczenie, zaśnięcie",NaN,NaN,0,0,1,0,2018-07-03 09:00:00
96060,2017-10-08,13:31:00,19*23'575,50*40'340,ZŁOTY POTOK,NaN,NaN,W793,NaN,Obszar niezabudowany,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,2,0,0,1,2017-10-08 13:31:00
74894,2017-05-30,00:56:00,19*06'198,50*50'155,CZĘSTOCHOWA,ŁÓDZKA,243,P6601S,NaN,Obszar zabudowany,...,Nie ma,NaN,NaN,NaN,Stan jezdni,0,0,1,0,2017-05-30 00:56:00
159419,2018-11-08,08:40:00,18*25'232,49*59'313,CZYŻOWICE,NOWA,51,P5017S,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-11-08 08:40:00
62960,2017-03-10,12:20:00,19*25'362,50*28'519,ZAWIERCIE,GEN. WŁADYSŁAWA SIKORSKIEGO,6,P1701S,NaN,Obszar zabudowany,...,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2017-03-10 12:20:00
121602,2018-03-13,15:15:00,19*00'594,50*22'011,WOJKOWICE,SOBIESKIEGO,368,I2,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018-03-13 15:15:00
58238,2017-02-06,09:17:00,18*45'366,49*59'439,SUSZEC,WYZWOLENIA,46,P4100S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0,2017-02-06 09:17:00
104783,2017-11-27,17:55:00,18*55'309,50*16'241,ŚWIĘTOCHŁOWICE,LIPOWA,52,GMINNA,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2017-11-27 17:55:00
151747,2018-09-25,16:23:00,19*08'225,50*18'033,SOSNOWIEC,NaN,NaN,94,NaN,Obszar niezabudowany,...,Jest,2.0,Inne przyczyny,NaN,NaN,0,0,1,0,2018-09-25 16:23:00


In [11]:
# Data zdarzenia i Godzina do typu datetime
df['data_full'] = pd.to_datetime(df['data_full'], format='%Y-%m-%d %H:%M:%S')
df.sample(10)
# TO-DO: Jaki to dzien tygodnia? Jaki to dzien miesiaca? To dzien czy noc? Jaka faza ksiezyca itd..

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
9568,2016-03-08,12:00:00,18*59'359,50*14'438,KATOWICE,POLA,32,P6501S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprawidłowe: cofanie,NaN,NaN,0,0,1,0,2016-03-08 12:00:00
135471,2018-06-12,18:46:00,19*04'412,50*14'233,KATOWICE,SZOPIENICKA,43,P6485S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-06-12 18:46:00
171955,2019-01-19,10:20:00,19*08'099,50*18'240,BĘDZIN,27 STYCZNIA,NaN,GMINNA,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2019-01-19 10:20:00
5835,2016-02-17,13:00:00,18*58'400,50*12'320,KATOWICE,GEN.ZYGMUNTA JANKEGO,150,P6442S,NaN,Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odl. między pojazdami,NaN,NaN,0,0,1,0,2016-02-17 13:00:00
57938,2017-02-01,15:05:00,18*59'032,49*48'311,BIELSKO-BIAŁA,TWÓRCZA,11,G160901S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2017-02-01 15:05:00
131728,2018-05-18,10:15:00,19*00'314,50*47'050,WYRAZÓW,CZĘSTOCHOWSKA,NaN,46,NaN,Obszar zabudowany,...,Jest,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018-05-18 10:15:00
1342,2016-01-14,09:30:00,19*00'580,50*15'390,KATOWICE,SŁOWACKIEGO,12,P6518S,NaN,Obszar zabudowany,...,Jest,NaN,Inne przyczyny (od 11.2015),NaN,NaN,0,0,1,0,2016-01-14 09:30:00
114415,2018-01-24,16:10:00,19*06'327,50*18'592,BĘDZIN,NaN,NaN,86,"15,1",Obszar niezabudowany,...,Jest,4.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2018-01-24 16:10:00
50693,2016-12-16,14:29:00,19*28'208,50*28'399,ZAWIERCIE,IGNACEGO PADEREWSKIEGO,NaN,78,NaN,Obszar zabudowany,...,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-12-16 14:29:00
205435,2019-08-14,07:40:00,18*55'302,50*29'414,MIASTECZKO ŚLĄSKIE,WOŹNICKA,NaN,W908,NaN,Obszar zabudowany,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,1,0,0,1,2019-08-14 07:40:00


#### Wspolrzedne geograficzne

Domyslny zapis wspolrzednych to D\*M'S([D]egree, [M]inute, [S]cond) np. 18\*53'289. Jednak, gdy pierwsza cyfra sekund jest zero, czasem wystepuje zapis D\*M''S np. 18\*53''11. Wystepowaly takze bledne zapisy takie jak 4\*\*56'569 i 50\*09'\*01.

In [12]:
def dmsToDecimal(coordinate_as_string):
    try:
        # Zapis domyslny to D*M'S([D]egree, [M]inute, [S]cond) zeby sprawnie podzielic kazda wspolrzedna na liste trzyelementowa
        # zamieniam ' na *, a nastepnie dziele orzymany rezultat znakami *. W trakcie konwersji danych napotkalam rozne nieprawidlowe
        # zapisy opisane w komorce powyzej. Wszystkie wystepujace nietypowe znaki takie jak **, '* i '' zamieniam na * i dziele znakiem *
        degree, minute, second = str(coordinate_as_string).replace("**", "*").replace("'*", "*").replace("''", "*").replace("'", '*').split("*")
    except:
        # Jesli nie dzieli na liste trzyelementowa to najprawdopodobniej trzeci element jest pusty(lista dwuelementowa).
        # Wtedy pierwszy element to stopien, drugi minuta. Sekundy zas sa rowne 0.
        degree, minute = str(coordinate_as_string).replace("**", "*").replace("'*", "*").replace("''", "*").replace("'", '*').split("*")
        second=0
    # W niektorych wypadkach udaje sie podzielic wsadowego stringa tak aby uzyskac ostatnia czesc jako pusty string.
    #Ponizej przeksztalcam pustego stringa w wartosc 0
    if second == "" or second == " ":
        second=0
    # Wlasciwe przeksztalcenie wspolrzednej z wartosci katowych na wartosc liczbowa
    coordinate_as_float = float(degree) + float(minute)/60 + float(second)/(60*60)
    return coordinate_as_float
# Przygotowana wyzej funkcje wywoluje na calych kolumnach dancych
df['GPS x'] = df['GPS x'].apply(lambda x: dmsToDecimal(x))
df['GPS y'] = df['GPS y'].apply(lambda x: dmsToDecimal(x))
df.sample(10)
# TO-DO Wyszukac i wykorzystac biblioteke okreslajaca wspolrzedne GPS na podstawie adresu

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
126797,2018-04-17,16:15:00,19.065556,50.584722,OŻAROWICE,TOPOLOWA,NaN,P3236S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-04-17 16:15:00
220023,2019-11-04,20:45:00,19.171944,50.821111,CZĘSTOCHOWA,ALEJA POKOJU,14,P6628S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2019-11-04 20:45:00
178190,2019-02-28,18:30:00,18.692778,50.303889,GLIWICE,RYBNICKA,207,I2,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2019-02-28 18:30:00
10005,2016-03-17,09:50:00,18.719444,49.766667,CIESZYN,BIELSKA,100,P2619S,NaN,Obszar zabudowany,...,Jest,2.0,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-03-17 09:50:00
123957,2018-03-29,10:40:00,19.130556,50.319722,SIEMIANOWICE ŚLĄSKIE,KS.JANA KAPICY,9A,P9620S,NaN,Obszar zabudowany,...,Jest,3.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-03-29 10:40:00
184027,2019-04-05,12:15:00,19.007222,50.319444,KATOWICE,ZBIGNIEWA CYBULSKIEGO,3,G100308S,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-04-05 12:15:00
79784,2017-06-29,13:35:00,18.859444,50.589722,POŁOMIA,PYSKOWICKA,NaN,P2905S,NaN,Obszar niezabudowany,...,Nie ma,2.0,NaN,NaN,Stan jezdni,0,0,1,0,2017-06-29 13:35:00
95342,2017-10-05,06:30:00,19.222778,50.317778,MYSŁOWICE,3 MAJA,7,P8800S,NaN,Obszar zabudowany,...,Jest,NaN,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2017-10-05 06:30:00
28844,2016-07-30,05:30:00,18.608611,50.012222,JASTRZĘBIE-ZDRÓJ,GÓRNICZA,1,G220129S,NaN,Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2016-07-30 05:30:00
98413,2017-10-22,13:25:00,18.731389,50.351111,GLIWICE,ŁUŻYCKA,12,P7211S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2017-10-22 13:25:00


#### Miejsce zdarzenia



In [13]:
# Zmiana nazwy kolumn na nazwy bez spacji i polskich znakow
# df.rename(columns={'Miejscowość': 'Miejscowosc', 'Numer domu': "NrDomu", "Numer drogi": "NrDrogi"}, inplace=True)
df.columns

Index(['Data zdarzenia', 'Godzina', 'GPS x', 'GPS y', 'Miejscowość', 'Ulica',
       'Numer domu', 'Numer drogi', 'KM HM (Pikietaż)', 'Obszar',
       'Skrzyżowanie', 'Geometria drogi', 'Charakterystyka miejsca zdarzenia',
       'Oświetlenie', 'Warunki atmosferyczne', 'Rodzaj zdarzenia', 'Wina',
       'Rodzaj drogi', 'Nawierzchnia drogi', 'Stan nawierzchni',
       'Sygnalizacja świetlna', 'Oznakowanie poziome',
       'Liczba pasów w zdarzeniu', 'Zachowanie kierującego',
       'Zachowanie pieszego', 'Inne przyczyny', 'Liczba rannych w zdarzeniu',
       'Liczba zabitych w zdarzeniu', 'Liczba kolizji', 'Liczba wypadków',
       'data_full'],
      dtype='object')

In [14]:
# Pikietaz okresla miejsce zdarzenia na drodze poza terenem zabudowanym
#Zapisywany jako xx,yyy gdzie:
#xx – całkowita liczba kilometrów od początku drogi;
#yyy – całkowita liczba metrów liczona od ostatniego pełnego kilometra.
# W zbiorze danych prawdopodobnie jako wartosc w km, tyle ze z uzyciem "," jako separatora dziesietnego
def pikietazToFloat(pikietazRaw):
    if isinstance(pikietazRaw, str):
        pikietazAsFloat = float(pikietazRaw.replace(",", "."))
    else:
        pikietazAsFloat = pikietazRaw
    return pikietazAsFloat
df['KM HM (Pikietaż)']=df['KM HM (Pikietaż)'].apply(lambda x: pikietazToFloat(x))
# df.rename(columns={'KM HM (Pikietaż)': 'KM_HM_Pikietaz'})
df.sample(10)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
108204,2017-12-15,20:24:39,18.824167,50.363056,GLIWICE,BŁ.CZESŁAWA,60,P7222S,NaN,Obszar zabudowany,...,Jest,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2017-12-15 20:24:39
198625,2019-07-02,12:20:00,19.250278,50.385000,MYSŁOWICE,KATOWICKA,75,I2,NaN,Obszar zabudowany,...,Nie ma,2.0,Inne przyczyny,NaN,NaN,0,0,1,0,2019-07-02 12:20:00
1058,2016-01-12,11:25:00,18.266111,50.114167,CYPRZANÓW,JANOWSKA,25,P3505S,NaN,Obszar zabudowany,...,Jest,NaN,NaN,Nieostrożne wejście na jezdnię: przed jadącym ...,NaN,0,0,1,0,2016-01-12 11:25:00
181289,2019-03-19,06:51:00,19.380278,50.477778,DĄBROWA GÓRNICZA,SZOSOWA,70,W796,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,1,0,0,1,2019-03-19 06:51:00
125340,2018-04-09,07:39:00,19.047222,50.353056,KATOWICE,NaN,NaN,A4,339.0,Obszar niezabudowany,...,Jest,8.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-04-09 07:39:00
115893,2018-02-04,09:05:00,18.818056,50.489722,TARNOWSKIE GÓRY,GLIWICKA,NaN,78,72.5,Obszar niezabudowany,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2018-02-04 09:05:00
15092,2016-04-26,07:50:00,19.074444,50.290000,KATOWICE,PANEWNICKA,NaN,P6494S,NaN,Obszar zabudowany,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-04-26 07:50:00
215661,2019-10-13,15:00:00,19.039167,49.815556,SZCZYRK,MYŚLIWSKA,43,W942,NaN,Obszar zabudowany,...,Nie ma,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-10-13 15:00:00
145140,2018-08-15,22:15:00,19.120278,50.368611,KATOWICE,WALENTEGO ROŹDZIEŃSKIEGO,NaN,79,NaN,Obszar zabudowany,...,Jest,5.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2018-08-15 22:15:00
187111,2019-04-24,14:50:00,18.798889,50.393611,GLIWICE,STANISŁAWA KONARSKIEGO,25,G130328S,NaN,Obszar zabudowany,...,Jest,3.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-04-24 14:50:00


In [15]:
# Obszar: zabudowany/niezabudowany
# Przypisane zostaly wartosci logiczne: Obszar zabudowany-True, Obszar niezabudowany False.
# W razie potrzeby takie wartosci mozna wygodnie przeksztalcic na int o wartosciach odpowiednio 1 i 0
df['Obszar'].value_counts()
df['Obszar'] = df['Obszar'].apply(lambda x: True if x=='Obszar zabudowany' else False)
df.sample(10)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,data_full
178204,2019-02-27,16:40:00,18.775278,50.076389,ŻORY,MIKOŁOWSKA,13,P9004S,NaN,True,...,Jest,2.0,NaN,NaN,Stan jezdni,0,0,1,0,2019-02-27 16:40:00
7653,2016-02-29,21:30:00,19.198889,50.253889,KATOWICE,GOSPODARCZA,NaN,P6433S,NaN,True,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-02-29 21:30:00
161185,2018-11-20,15:00:00,19.180556,50.895278,CZĘSTOCHOWA,DEKABRYSTÓW,NaN,P6611S,NaN,True,...,Jest,4.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-11-20 15:00:00
207267,2019-08-26,07:00:00,19.062778,50.265833,KATOWICE,MIKOŁOWSKA,52,P6479S,NaN,True,...,Jest,3.0,NaN,NaN,Nieustalone,0,0,1,0,2019-08-26 07:00:00
16089,2016-04-28,08:00:00,19.225556,50.965278,CZĘSTOCHOWA,RYNEK WIELUŃSKI,NaN,G110744S,NaN,True,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-04-28 08:00:00
103616,2017-11-21,13:40:00,18.643889,50.396389,GLIWICE,STAROMIEJSKA,NaN,P7211S,NaN,True,...,Jest,2.0,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0,2017-11-21 13:40:00
107877,2017-12-14,15:10:00,19.205833,50.344722,MYSŁOWICE,KACZA,4,G240006S,NaN,True,...,Nie ma,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2017-12-14 15:10:00
87010,2017-08-19,08:43:00,19.667222,50.792778,PODLESIE,NaN,31,P1098S,NaN,True,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2017-08-19 08:43:00
155972,2018-10-19,17:35:00,19.113889,50.284167,KATOWICE,WALENTEGO ROŹDZIEŃSKIEGO,93,86,NaN,True,...,Jest,4.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-10-19 17:35:00
23916,2016-06-22,17:55:00,18.729167,50.369167,ŻERNICA,JÓZEFA SZAFRANKA,NaN,P2916S,NaN,True,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,1,0,0,1,2016-06-22 17:55:00


In [16]:
# Skrzyzowanie
# Zmiana nazwy na niezawierajaca polskich znakow
df['Skrzyżowanie'].unique()
df['Skrzyżowanie'].value_counts()
# Zamiana wartosci tekstowych na int zgodnie z zasada:
# 0 : O ruchu okrężnym; 1 : Równorzędne; 2 : Z drogą z pierwsz.
skrzyzowaniemap = {'O ruchu okrężnym':0, 'Równorzędne':1, 'Z drogą z pierwsz.':2}
df=df.applymap(lambda x: skrzyzowaniemap.get(x) if x in skrzyzowaniemap else x)
# df.rename(columns={'Skrzyżowanie': 'Skrzyzowanie'})

In [17]:
df['Geometria drogi'].value_counts()
# Geometria pozioma: 
# 0: Odcinek prosty; 1: Zakret(luk)

# Geometria pionowa: 
# 0: Spadek; 1: wzniesienie; 2: wierzcholek wzniesienia



Odcinek prosty                      142703
Zakręt, łuk                          16917
Spadek                                1866
Wzniesienie                           1317
Odcinek prosty; Spadek                 430
Odcinek prosty; Wzniesienie            329
Zakręt, łuk; Spadek                    179
Zakręt, łuk; Wzniesienie               108
Wierzchołek wzn.                        50
Odcinek prosty; Wierzchołek wzn.        16
Zakręt, łuk; Wierzchołek wzn.            5
Odcinek prosty; Zakręt, łuk              1
Name: Geometria drogi, dtype: int64

In [27]:
#df['Geometria_drogi_poziom'] = df['Geometria drogi'].copy()
#df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 0 if 'Odcinek prosty' in str(x) else x)
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 1 if 'Zakręt' in str(x) else x)
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 1 if 'łuk' in str(x) else x)
# Co z reszta? Tzn z sytuacja kiedy nie jest zdefiniowane czy droga byla prosta czy byl to zakret?
df['Geometria_drogi_poziom'].value_counts()

0                   143479
1                    17209
Spadek                1866
Wzniesienie           1317
Wierzchołek wzn.        50
Name: Geometria_drogi_poziom, dtype: int64

In [28]:
df['Geometria_drogi_pion'] = df['Geometria drogi'].copy()
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 0 if 'Spadek' in str(x) else x)
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 1 if 'Wzniesienie' in str(x) else x)
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 2 if 'Wierzchołek wzn.' in str(x) else x)
# Co z reszta? Tzn z sytuacja kiedy nie jest zdefiniowane uksztaltowanie terenu?
df['Geometria_drogi_pion'].value_counts()

Odcinek prosty                 142703
Zakręt, łuk                     16917
0                                2475
1                                1754
2                                  71
Odcinek prosty; Zakręt, łuk         1
Name: Geometria_drogi_pion, dtype: int64